## Zahra's code

In [6]:
import numpy as np
import tifffile as tif
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import ball

In [4]:
#first, make a map of cells
pth = "/jukebox/wang/zahra/h129_contra_vs_ipsi/pma_to_aba/20180410_jg52_bl6_lob7_05/posttransformed_zyx_voxels.npy"
converted_points = np.load(pth)

zyx = np.asarray([(int(xx[0]), int(xx[1]), int(xx[2])) for xx in converted_points]) #cells are counted in horizontal volumes

In [7]:
#read registered volume
vol = "/jukebox/wang/zahra/h129_contra_vs_ipsi/reg_to_allen/20180410_jg52_bl6_lob7_05/cell_to_reg/result.tif"
img = tif.imread(vol)  

In [8]:
#init empty vol 
cell_map = np.zeros_like(img).astype('uint16')

In [9]:
#fill map
for z,y,x in zyx:
    try:
        cell_map[z-1:z+1,y,x] = 1 #z dilation
    except Exception as e:
        print(e)

index 530 is out of bounds for axis 1 with size 528
index 529 is out of bounds for axis 1 with size 528
index 531 is out of bounds for axis 1 with size 528
index 530 is out of bounds for axis 1 with size 528
index 529 is out of bounds for axis 1 with size 528
index 531 is out of bounds for axis 1 with size 528
index 529 is out of bounds for axis 1 with size 528
index 528 is out of bounds for axis 1 with size 528
index 529 is out of bounds for axis 1 with size 528
index 530 is out of bounds for axis 1 with size 528
index 550 is out of bounds for axis 1 with size 528
index 555 is out of bounds for axis 1 with size 528
index 555 is out of bounds for axis 1 with size 528
index 562 is out of bounds for axis 1 with size 528
index 545 is out of bounds for axis 1 with size 528
index 548 is out of bounds for axis 1 with size 528
index 550 is out of bounds for axis 1 with size 528
index 554 is out of bounds for axis 1 with size 528
index 556 is out of bounds for axis 1 with size 528
index 562 is

index 541 is out of bounds for axis 1 with size 528
index 541 is out of bounds for axis 1 with size 528
index 547 is out of bounds for axis 1 with size 528
index 548 is out of bounds for axis 1 with size 528
index 548 is out of bounds for axis 1 with size 528
index 549 is out of bounds for axis 1 with size 528
index 549 is out of bounds for axis 1 with size 528
index 533 is out of bounds for axis 1 with size 528
index 532 is out of bounds for axis 1 with size 528
index 532 is out of bounds for axis 1 with size 528
index 533 is out of bounds for axis 1 with size 528
index 535 is out of bounds for axis 1 with size 528
index 535 is out of bounds for axis 1 with size 528
index 536 is out of bounds for axis 1 with size 528
index 538 is out of bounds for axis 1 with size 528
index 538 is out of bounds for axis 1 with size 528
index 540 is out of bounds for axis 1 with size 528
index 542 is out of bounds for axis 1 with size 528
index 543 is out of bounds for axis 1 with size 528
index 550 is

In [10]:
#apply x y dilation
r = 2
selem = ball(r)[int(r/2)]
# cell_map = cell_map.astype("uint8")
cell_map = np.asarray([cv2.dilate(cell_map[i], selem, iterations = 1) for i in range(cell_map.shape[0])])

In [14]:
tif.imsave("/home/ahoag/ngdemo/data/registered_cellmap_test.tif", cell_map)

In [2]:
# Now create the cloudvolume for this tif file
import os
import numpy as np

from concurrent.futures import ProcessPoolExecutor

from taskqueue import LocalTaskQueue
import igneous.task_creation as tc


from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
from PIL import Image

In [7]:
cell_map_filename = "/home/ahoag/ngdemo/data/registered_cellmap_test.tif"
cell_map = tif.imread(cell_map_filename)
cell_map.shape # z,y,x

(456, 528, 320)

In [10]:
home_dir = '/home/ahoag/ngdemo'
progress_dir = mkdir(home_dir + '/label_progress_regcells/')
def make_info_file():
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # can pick any popular uint
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = [ 40000, 40000, 40000 ], # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 1 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
        volume_size = [320,528,456], # X,Y,Z size in voxels
    )

    # If you're using amazon or the local file system, you can replace 'gs' with 's3' or 'file'
    vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/demo_dataset/registered_cells', info=info)
    vol.provenance.description = "A demo to show cells registered to the allen atlas"
    vol.provenance.owners = ['ahoag@princeton.edu'] # list of contact email addresses

    vol.commit_info() # generates gs://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates gs://bucket/dataset/layer/provenance json file
    return vol

In [11]:
def process_slice(z):
    print('Processing slice z=',z)
    
    array = cell_map[z-1].reshape((1,y_dim,x_dim)).T

    cloud_vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))

In [12]:
 cloud_vol = make_info_file()
""" tifffile is already loaded """
# image = np.array(tifffile.imread(brainvolume_file),dtype=np.uint16, order='F') # F stands for fortran order
z_dim,y_dim,x_dim = cell_map.shape

done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(cloud_vol.bounds.minpt.z, cloud_vol.bounds.maxpt.z)) # 1 indexed 

to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
# print("Remaining slices to upload are:",to_upload)

with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(process_slice, to_upload)

Remaining slices to upload are: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

Uploading: 100%|██████████| 1/1 [00:00<00:00, 72.94it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s].85it/s]

Processing slice z= 9





Uploading: 100%|██████████| 1/1 [00:00<00:00, 74.70it/s]

Processing slice z= 10


...

Uploading: 100%|██████████| 1/1 [00:00<00:00, 44.59it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 74.36it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00, 85.72it/s]



Processing slice z= 352
Processing slice z= 353
Processing slice z= 354


Processing slice z= 355


Processing slice z= 356


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
# make the igenous mesh to display in 3D
def make_mesh():
    # Mesh on 8 cores, use True to use all cores
    cloudpath = cloud_vol.cloudpath
    with LocalTaskQueue(parallel=8) as tq:
      tasks = tc.create_meshing_tasks(cloudpath, mip=0, shape=(256, 256, 256))
      tq.insert_all(tasks)
      tasks = tc.create_mesh_manifest_tasks(cloudpath)
      tq.insert_all(tasks)
    print("Done!")

In [14]:
make_mesh()

Downloading:  61%|██████▏   | 158/257 [00:00<00:00, 1575.92it/s]

Downloading:  99%|█████████▉| 254/257 [00:00<00:00, 2535.38it/s]
Downloading: 506it [00:00, 5053.22it/s]             

Downloading: 222it [00:00, 676.85it/s]                          

Downloading: 436it [00:00, 1715.09it/s]                         

Downloading: 316it [00:00, 1060.70it/s]                         

Downloading: 492it [00:00, 901.68it/s]                         

Downloading: 420it [00:00, 1711.73it/s]                         

Downloading: 508it [00:00, 2465.02it/s]                         

Downloading:  55%|█████▌    | 110/200 [00:00<00:00, 1092.84it/s]
Downloading: 314it [00:00, 863.74it/s]                         

Downloading:  89%|████████▉ | 178/200 [00:00<00:00, 870.64it/s] 
Downloading: 356it [00:00, 1007.16it/s]                        

Downloading:  81%|████████  | 162/200 [00:00<00:00, 738.86it/s]
Downloading: 382it [00:00, 662.10it/s]                         

Downloading: 324it [00:00, 863.

Done!



Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ahoag/anaconda3/envs/ng/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent

In [15]:
# Start a viewer and see what it looks like
import neuroglancer
from collections import OrderedDict
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [16]:
# First need to host the cloudvolume -- did that in the terminal at port 1337

In [18]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
    )

#     s.layers[0]._json_data['segments']=unique_segments
print(viewer)

http://127.0.0.1:36673/v/79371bfafd40b192451b07aae07ade6d2c02dccb/


I loaded the allen atlas on there as well and it looks like the atlas is offset in z by a few hundred planes. Let's look at the allen atlas info file to see what is going on

In [30]:
allen_vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/atlas/allenatlas/')

In [31]:
allen_vol.info

{'data_type': 'uint32',
 'mesh': 'mesh_mip_0_err_40',
 'num_channels': 1,
 'scales': [{'chunk_sizes': [[1024, 1024, 1]],
   'encoding': 'raw',
   'key': '40000_40000_40000',
   'resolution': [40000, 40000, 40000],
   'size': [320, 528, 456],
   'voxel_offset': [0, 0, 1]}],
 'type': 'segmentation'}

Yeah, looks like the x and z dimensions are swapped. Just remake the allen atlas with the correct dimensions and it should be fine. 

In [19]:
ss = neuroglancer.ScreenshotSaver(viewer, '/home/ahoag/ngdemo/screenshots/')
ss.capture()  

KeyboardInterrupt: 